In [3]:
import pandas as pd
xmlfile = pd.read_xml("/Volumes/Samsung_T5/argo/deltas.dfxml")


In [6]:
xmlfile

,type,execution_environment,image_filename,partition_offset,block_size,ftype,ftype_str,block_count,first_block,last_block,original_volume,fileobject
0,Disk image difference set,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,None,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,None,NaN,/Volumes/Storage/mtlaam/tmp/workflow/diskprint...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
3,None,NaN,None,105906176.0,4096.0,1.0,ntfs,5216767.0,0.0,5216766.0,NaN,NaN
4,None,NaN,None,1048576.0,4096.0,1.0,ntfs,25599.0,0.0,25598.0,NaN,NaN


In [1]:
import dfxml
iters = dfxml.iter_dfxml('/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.xml', preserve_elements=True)
obj_list = []
for obj in iters:
    obj_list.append(obj)

In [3]:
print(len(obj_list))

49745


In [21]:
a = obj_list[34485] #28982

In [22]:
a.filename()

'WINDOWS/system32/dllcache/fpexedll.dll'

In [13]:
print(dir(a))

['TIMETAGLIST', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_byte_runs', '_tags', 'allocated', 'allocated_inode', 'allocated_name', 'atime', 'byte_runs', 'compressed', 'content_for_run', 'contents', 'crtime', 'ctime', 'dtime', 'encrypted', 'ext', 'file_present', 'filename', 'filesize', 'frag_start_sector', 'fragments', 'gid', 'has_contents', 'has_sector', 'has_tag', 'hashdigest', 'imagefile', 'inode', 'is_dir', 'is_file', 'is_virtual', 'libmagic', 'md5', 'meta_type', 'mode', 'mtime', 'name_type', 'partition', 'savefile', 'sha1', 'sha256', 'tag', 'tempfile', 'times', 'uid', 'volume', 'xml_element']


In [23]:
a.byte_runs()

In [24]:
print(dir(a.byte_runs()[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'decode_sax_attributes', 'decode_xml_attributes', 'extra_len', 'file_offset', 'fs_offset', 'has_sector', 'hashdigest', 'img_offset', 'len', 'sector_count', 'sector_size', 'start_sector', 'uncompressed_len']


In [51]:
b = a.byte_runs()[0]


In [54]:
c = vars(b)

In [56]:
print(type(c))
print(c)

<class 'dict'>
{'img_offset': 5673254400, 'file_offset': 0, 'len': None, 'sector_size': 512, 'hashdigest': {}, 'fs_offset': 5673222144, 'uncompressed_len': 8192}


In [19]:
iterlist = (row for row in iters)
print(iterlist)

<generator object <genexpr> at 0x7fd4a7347b30>


In [36]:
newlist= [obj for obj in iters]

In [37]:
newlist

[]

In [41]:
for what in iterlist:
    print(what)

In [9]:
# MPIpool.py

from mpi4py.futures import MPIPoolExecutor
import math
import textwrap
import hashlib
import os
import pandas as pd
from itertools import repeat
import sqlite3
import numpy as np
import pprint
from pprint import pprint
import dfxml 

def compute_hash(range_tuple):
    img_path = '/home/oadegbeh/M57/pat-2009-12-10.raw.raw'
    sector_hash_list = []
    print(range_tuple[0], range_tuple[1])
    sector_size = 512
    img_h = open(img_path, 'rb')
    for pointer in range(range_tuple[0], range_tuple[1]):
                #print(pointer)
                img_offset = pointer*sector_size
                img_h.seek(img_offset)
                fsector = img_h.read(sector_size)
                sector_md5 = hashlib.md5(fsector).hexdigest()
                sector_hash_list.append((img_offset,sector_md5))
    return sector_hash_list

def process_object_sectors(range_tuple):
    sector_size = 512
    img_csv = "/Volumes/Samsung_T5/argo/pat2.csv"
    img_csv_df = pd.read_csv(img_csv)
    global options
    files_cols = ['obj_id', 'partition','inode','filename','filesize']
    block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
    file_df = pd.DataFrame(columns=files_cols)
    ebyterun_df = pd.DataFrame(columns=block_hashes_cols)
    return_list = []
    for obj in range(range_tuple[0], range_tuple[1]):
        # Filter out specific filenames create by TSK that are not of use
        print("processing this  %s" % str(obj))
        print(type(obj))
        #if int(obj._tags['id']) == id:# and obj.filesize() == False:
        #counter += 1
        #print(obj.filename())
        data = [str(obj), obj.partition(),obj.inode(), obj.filename(),  obj.filesize()] #obj._tags['id']
        file_df.loc[len(file_df.index)] = data
        print(data)
        byterun = []
        persist_img_offset = 0
        persist_fs_offset = 0
        persist_file_offset = 0
        remaining_len = obj.filesize()
        #ebyterun_df = pd.DataFrame(columns=['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1'])
        for ebyterun in obj.byte_runs():
            #pprint(vars(ebyterun)) 
            #byterun.append([ebyterun.img_offset,ebyterun.file_offset, ebyterun.len, ebyterun.fs_offset])
            byterun.append(ebyterun.file_offset)
            persist_file_offset = ebyterun.file_offset
            byterun.append(ebyterun.len)
            if hasattr(ebyterun,'img_offset'): 
                byterun.append(ebyterun.img_offset)
                if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
            if hasattr(ebyterun,'fs_offset'): 
                byterun.append(ebyterun.fs_offset)
                if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset
            if hasattr(ebyterun,'len'): 
                byterun.append(ebyterun.len)
                if ebyterun.len != None: persist_len = ebyterun.len
            #print("img_offset", img_offset, "file offset", ebyterun.file_offset)
            #print("file_offset", "len", "img_offset", "fs_offset")
            byterun_start = int(persist_img_offset / sector_size)
            byterun_end = int((math.ceil((persist_img_offset + ebyterun.len) / sector_size))) - 1
            len_run = np.arange(persist_len,0,-sector_size) #remaining_len
            sector_run = np.full(len(len_run), sector_size)
            len_run = np.minimum(len_run,sector_run)
            #len_run = min(sector_size,len_run.all())
            #print("len_run",len(len_run), len(len_run)*sector_size)
            #print("byterun_start", byterun_start,persist_img_offset, "byterun_end", byterun_end,  "len", ebyterun.len)
            #print("ebyterun_df", ebyterun_df.shape, "img_csv", img_csv_df.loc[byterun_start:byterun_end,'img_offset'].shape )
            #print(img_csv_df.loc[byterun_start:byterun_end,:])
            #print("img_csv_len",len(img_csv_df.loc[byterun_start:byterun_end, :]))
            ebyterun_df.loc[:,'img_sector_offset'] = np.arange(len(img_csv_df))
            ebyterun_df.loc[byterun_start:byterun_end,'obj_id'] = str(obj) #obj._tags['id']
            ebyterun_df.loc[byterun_start:byterun_end,'img_offset'] = img_csv_df.loc[byterun_start:byterun_end,'img_offset']
            ebyterun_df.loc[byterun_start:byterun_end,'fs_offset'] = np.arange(persist_fs_offset,persist_fs_offset+ebyterun.len,sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'file_offset'] = np.arange(persist_file_offset,persist_file_offset+ebyterun.len, sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'len'] = len_run
            #print("remaining_len",remaining_len)
            #remaining_len-=sector_size
            ebyterun_df.loc[byterun_start:byterun_end,'md5'] = img_csv_df.loc[byterun_start:byterun_end,'md5']
            return_list.append((file_df,ebyterun_df))
        print("done processing %s" % str(obj))
    return return_list   

def determine_subranges(fullrange, num_subranges):
    """
    Break fullrange up into smaller sets of ranges that cover all
    the same numbers.
    """
    subranges = []
    inc = fullrange[1] // num_subranges
    for i in range(fullrange[0], fullrange[1], inc):
        subranges.append( (i, min(i+inc, fullrange[1])) )
    return( subranges )


if __name__ == '__main__':
    #sector_size = 512
    img_path = '/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.xml'
    object_number = dfxml.iter_dfxml(img_path, preserve_elements=True)
    img_objcount = len(list(object_number))
    fullrange = (0, img_objcount)
    num_subranges = 1000
    subranges = determine_subranges(fullrange, num_subranges)

    executor = MPIPoolExecutor()
    sectors_df_list = executor.map(process_object_sectors, subranges)
    #files_df_list = executor.map(process_object_files, subranges)
    executor.shutdown()
    grand_sectors_df = pd.DataFrame()
    grand_files_df = pd.DataFrame()
    # flatten the list of lists
    for df in sectors_df_list:
        grand_files_df = pd.concat([grand_files_df, df[0]])
        grand_sectors_df = pd.concat([grand_sectors_df, df[1]])
    #for df in files_df_list:
    #    grand_files_df = pd.concat([grand_files_df, df])
    #print(textwrap.fill(str(primes),80))
    con = sqlite3.connect("/Volumes/Samsung_T5/M57/pat.db")
    grand_sectors_df.to_sql('block_hashes', con, index=True)
    grand_files_df.to_sql('files', con, index=True)


In [ ]:
#LATEST VERSION
# MPIpool.py

from mpi4py.futures import MPIPoolExecutor
import math
import textwrap
import hashlib
import os
import pandas as pd
from itertools import repeat
#import sqlite3
import numpy as np
import pprint
from pprint import pprint
import dfxml 

def compute_hash(range_tuple):
    img_path = '/home/oadegbeh/M57/pat-2009-12-10.raw.raw'
    sector_hash_list = []
    print(range_tuple[0], range_tuple[1])
    sector_size = 512
    img_h = open(img_path, 'rb')
    for pointer in range(range_tuple[0], range_tuple[1]):
                #print(pointer)
                img_offset = pointer*sector_size
                img_h.seek(img_offset)
                fsector = img_h.read(sector_size)
                sector_md5 = hashlib.md5(fsector).hexdigest()
                sector_hash_list.append((img_offset,sector_md5))
    return sector_hash_list

def process_object_sectors(objlist):
    sector_size = 512
    img_csv = "/home/oadegbeh/M57/pat2.csv"
    img_csv_df = pd.read_csv(img_csv)
    global options
    files_cols = ['obj_id', 'partition','inode','filename','filesize']
    block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
    file_df = pd.DataFrame(columns=files_cols)
    ebyterun_df = pd.DataFrame(columns=block_hashes_cols)
    return_list = []
    for obj in objlist:#range(range_tuple[0], range_tuple[1]):
        # Filter out specific filenames create by TSK that are not of use
        print("processing %s" % obj._tags['id']) #str(obj)
        #print(type(obj))
        #if int(obj._tags['id']) == id:# and obj.filesize() == False:
        #counter += 1
        #print(obj.filename())
        data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()] #
        file_df.loc[len(file_df.index)] = data
        print(data)
        byterun = []
        persist_img_offset = 0
        persist_fs_offset = 0
        persist_file_offset = 0
        remaining_len = obj.filesize()
        #ebyterun_df = pd.DataFrame(columns=['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1'])
        for ebyterun in obj.byte_runs():
            #pprint(vars(ebyterun)) 
            #byterun.append([ebyterun.img_offset,ebyterun.file_offset, ebyterun.len, ebyterun.fs_offset])
            byterun.append(ebyterun.file_offset)
            persist_file_offset = ebyterun.file_offset
            byterun.append(ebyterun.len)
            if hasattr(ebyterun,'img_offset'): 
                byterun.append(ebyterun.img_offset)
                if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
            if hasattr(ebyterun,'fs_offset'): 
                byterun.append(ebyterun.fs_offset)
                if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset
            if hasattr(ebyterun,'len'): 
                byterun.append(ebyterun.len)
                if ebyterun.len != None: persislen = ebyterun.len
            #print("img_offset", img_offset, "file offset", ebyterun.file_offset)
            #print("file_offset", "len", "img_offset", "fs_offset")
            byterun_start = int(persist_img_offset / sector_size)
            byterun_end = int((math.ceil((persist_img_offset + ebyterun.len) / sector_size))) - 1
            len_run = np.arange(persist_len,0,-sector_size) #remaining_len
            sector_run = np.full(len(len_run), sector_size)
            len_run = np.minimum(len_run,sector_run)
            #len_run = min(sector_size,len_run.all())
            #print("len_run",len(len_run), len(len_run)*sector_size)
            #print("byterun_start", byterun_start,persist_img_offset, "byterun_end", byterun_end,  "len", ebyterun.len)
            #print("ebyterun_df", ebyterun_df.shape, "img_csv", img_csv_df.loc[byterun_start:byterun_end,'img_offset'].shape )
            #print(img_csv_df.loc[byterun_start:byterun_end,:])
            #print("img_csv_len",len(img_csv_df.loc[byterun_start:byterun_end, :]))
            ebyterun_df.loc[:,'img_sector_offset'] = np.arange(len(img_csv_df))
            ebyterun_df.loc[byterun_start:byterun_end,'obj_id'] = str(obj) #obj._tags['id']
            ebyterun_df.loc[byterun_start:byterun_end,'img_offset'] = img_csv_df.loc[byterun_start:byterun_end,'img_offset']
            ebyterun_df.loc[byterun_start:byterun_end,'fs_offset'] = np.arange(persist_fs_offset,persist_fs_offset+ebyterun.len,sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'file_offset'] = np.arange(persist_file_offset,persist_file_offset+ebyterun.len, sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'len'] = len_run
            #print("remaining_len",remaining_len)
            #remaining_len-=sector_size
            ebyterun_df.loc[byterun_start:byterun_end,'md5'] = img_csv_df.loc[byterun_start:byterun_end,'md5']
            return_list.append((file_df,ebyterun_df))
        print("done processing %s" % obj._tags['id']) #str(obj)
    return return_list   

def determine_subranges(obj_list, num_subranges):#fullrange
    """
    Break fullrange up into smaller sets of ranges that cover all
    the same numbers.
    """
    subranges = []
    inc =  len(obj_list)// num_subranges #fullrange[1]
    for i in range(0, len(obj_list), inc):#fullrange[0], fullrange[1]
        subranges.append( (obj_list[i], obj_list[min(i+inc,len(obj_list)-1 )]) )#fullrange[1]
    return( subranges )


if __name__ == '__main__':
    #sector_size = 512
    img_path = '/home/oadegbeh/M57/pat-2009-12-10.raw.xml'
    object_gen = dfxml.iter_dfxml(img_path, preserve_elements=True)
    obj_list = []
    for each_obj in object_gen:
        obj_list.append(each_obj)
    #img_objcount = len(obj_list)
    #fullrange = (0, img_objcount)
    num_subranges = 1000
    subranges = determine_subranges(obj_list, num_subranges)#fullrange

    executor = MPIPoolExecutor()
    sectors_df_list = executor.map(process_object_sectors, subranges)
    #files_df_list = executor.map(process_object_files, subranges)
    executor.shutdown()
    grand_sectors_df = pd.DataFrame()
    grand_files_df = pd.DataFrame()
    # flatten the list of lists
    for df in sectors_df_list: #COULD THIS BE A PROBLEM POINT?
        grand_files_df = pd.concat([grand_files_df, df[0]])
        grand_sectors_df = pd.concat([grand_sectors_df, df[1]])
    #for df in files_df_list:
    #    grand_files_df = pd.concat([grand_files_df, df])
    #print(textwrap.fill(str(primes),80))
    con = sqlite3.connect("/scratch/oadegbeh/pat.db")
    grand_sectors_df.to_sql('block_hashes', con, index=True)
    grand_files_df.to_sql('files', con, index=True)
